# Ejercicios clase 4


## Análisis Frecuentista con Python
Realizar el análisis frecuentista en el grupo 1D. Calcular la diferencia de las tasas de conversión, realizar un Test de proporciones para analizar si el efecto es significativo y aplicar el estimador WALD para compensar el ATE.

In [ ]:
import numpy as np
from scipy.stats import chi2_contingency

def calcular_diferencia_conversion_rate(dataset):
  control = dataset[dataset["GRUPO"] == "GC"]
  grupo_1D = dataset[dataset["GRUPO"] == "1D"]

  conversion_rate_control = control["Outcome"].mean()
  conversion_rate_1D = grupo_1D["Outcome"].mean()

  ATE = conversion_rate_1D - conversion_rate_control
  return ATE

def test_proporciones(dataset):
  control = dataset[dataset["GRUPO"] == "GC"]
  grupo_1D = dataset[dataset["GRUPO"] == "1D"]

  conversiones_control = control["Outcome"].sum()
  conversiones_1D = grupo_1D["Outcome"].sum()

  no_conversiones_control = len(control) - conversiones_control
  no_conversiones_1D = len(grupo_1D) - conversiones_1D

  contingencia = np.array([[conversiones_control, no_conversiones_control],
                          [conversiones_1D, no_conversiones_1D]])

  chi2, p_value, dof, ex = chi2_contingency(contingencia)
  return p_value

def estimador_wald(dataset):
  control = dataset[dataset["GRUPO"] == "GC"]
  grupo_1D = dataset[dataset["GRUPO"] == "1D"]

  conversion_rate_control_shown = control[control["shown"] == 1]["Outcome"].mean()
  conversion_rate_1D_shown = grupo_1D[grupo_1D["shown"] == 1]["Outcome"].mean()

  ATE_WALD = (conversion_rate_1D_shown - conversion_rate_control_shown) / grupo_1D["shown"].mean()
  return ATE_WALD


## Estimar efecto de largo plazo con S Learner
Estimar efecto de largo plazo utilizando la técnica del S learner (con Regresión Lineal) para el grupo 1D.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

def efecto_largo_plazo_s_learner(dataset):
    features = ["first_purchase_window", "freq_rfm", "recency", "cant_dias_active", "recency_date"]

    control = dataset[dataset["GRUPO"] == "GC"]
    grupo_1D = dataset[dataset["GRUPO"] == "1D"]

    X_control = control[features]
    y_control = control["long_Outcome"]

    X_1D = grupo_1D[features]
    y_1D = grupo_1D["long_Outcome"]

    X = np.concatenate((X_control, X_1D), axis=0)
    y = np.concatenate((y_control, y_1D), axis=0)
    tratamiento = np.concatenate((np.zeros_like(y_control), np.ones_like(y_1D)), axis=0)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_stacked = np.hstack((X, tratamiento.reshape(-1, 1)))

    reg = LinearRegression().fit(X_stacked, y)
    coef_tratamiento = reg.coef_[-1]

    return coef_tratamiento